In [1]:
import numpy as np
import skimage.io
from skimage import restoration
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim
import os
import matplotlib.pyplot as plt

In [2]:
def tv_filter_output(gt_path, noisy_path, noise_factor, w, result_path):
        
    nf = str(noise_factor).replace(".", "")
    psnr_total  = 0
    ssim_total  = 0
    
    for root, dirs, files in os.walk(gt_path, topdown=False):
        for name in files:
            img = skimage.io.imread(gt_path + name)
            
            noisy_img = skimage.io.imread(noisy_path + nf + '/' + nf + '_' + name)
            noisy_img_normalized = noisy_img/255.0
            out = restoration.denoise_tv_chambolle(noisy_img_normalized, weight=w)
            out = np.clip(out, 0,1)*255
            out = out.astype('uint8')
            
            psnr_total += compare_psnr(img, out)
            ssim_total += compare_ssim(img, out, channel_axis=2)
            
            skimage.io.imsave(result_path + 'tv_' + name, out)
        
    psnr_total /= len(files)
    ssim_total /= len(files)
    
    print("PSNR: ", psnr_total)
    print("SSIM: ", ssim_total)
    
    return psnr_total, ssim_total

In [3]:
noise_factor = 0.3
nf = str(noise_factor).replace(".", "")
w = 0.1

In [4]:
mri_test_path = './data/mri_images_gt/bt_gt_test/'
ct_path = './data/ct_images_gt/'
lympho_path = './data/lymphocytes_images_gt/'

mri_test_path_noisy = mri_test_path + '../bt_noisy_test/'
ct_path_noisy = ct_path + '../ct_noisy/'
lympho_path_noisy = lympho_path + '../lymphocytes_noisy/'

result_path_mri = './data/results/tv/mri_test/' + nf + '/'
result_path_ct = './data/results/tv/ct_test/'  + nf + '/'
result_path_lympho = './data/results/tv/lympho_test/' + nf + '/'

os.makedirs(result_path_mri, exist_ok=True)
os.makedirs(result_path_ct, exist_ok=True)
os.makedirs(result_path_lympho, exist_ok=True)

In [5]:
psnr, ssim = tv_filter_output(mri_test_path, mri_test_path_noisy, noise_factor, w, result_path_mri)

PSNR:  19.688960014750453
SSIM:  0.3045320647503974


In [6]:
psnr, ssim = tv_filter_output(ct_path, ct_path_noisy, noise_factor, w, result_path_ct)

PSNR:  20.49582172331558
SSIM:  0.29039168104845386


In [7]:
psnr, ssim = tv_filter_output(lympho_path, lympho_path_noisy, noise_factor, w, result_path_lympho)

PSNR:  17.88188735908707
SSIM:  0.6184338560643486
